```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Static Layer](#intermediate-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNpdT71uAjEMfpXIE0jQB7ih0lUwINGFsDVV5UvMXcTlRz6HiiLevWmhA_Vk-fv1BWxyBA0cxvRpB2RRarszUdWZStcz5kFtYi6itngmviGaZ-1XYXoZUzdXy6Uy8Nrq_Xq3avftxwnZY5TpyU4nAxV-VvptNgn2Pvbz95sFRfcvRQuKtw8xv9pN1ZbubqoC5vxoAwsIxAG9q29cfs4GZKBABpq6OuSjAROvlVeyQ6G185IYmgOOEy0AiyR9jhYa4UJ_pJXHWivcWddvOI1hAA)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNpdT71uAjEMfpXIE0jQB7ih0lUwINGFsDVV5UvMXcTlRz6HiiLevWmhA_Vk-fv1BWxyBA0cxvRpB2RRarszUdWZStcz5kFtYi6itngmviGaZ-1XYXoZUzdXy6Uy8Nrq_Xq3avftxwnZY5TpyU4nAxV-VvptNgn2Pvbz95sFRfcvRQuKtw8xv9pN1ZbubqoC5vxoAwsIxAG9q29cfs4GZKBABpq6OuSjAROvlVeyQ6G185IYmgOOEy0AiyR9jhYa4UJ_pJXHWivcWddvOI1hAA)

Required steps:

1. **[Source]()**: contains the `"subvariant masterdata"` data collected to be used to mapping the COVID-19 variants and used for further processing.
2. **[Staging](#input-layer)**: inserts the raw `Source` data within the `[VWSSTAGE].[RIVM_SUBVARIANT_MAPPING]` table.
3. **[Subvariant Mapping](#static-layer)**: inserts the `Staging` data and cast the correct `Database Types` within `[VWSSTATIC].[RIVM_SUBVARIANT_MAPPING]` using business logic implemented in `[dbo].[SP_INSERT_SL_RIVM_SUBVARIANT_MAPPING]`.

- **<font color=teal>SL</font>**: Static Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Schemas.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[RIVM_SUBVARIANT_MAPPING]'))
CREATE TABLE [VWSSTAGE].[RIVM_SUBVARIANT_MAPPING] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE(),
    [VARIANT_CODE] [VARCHAR] (100) NULL,
    [VARIANT_NAME] [VARCHAR] (100) NULL,
    [LABEL] [VARCHAR] (100) NULL,
    [HISTORICALY_SIGNIFICANT] [VARCHAR] (100) NULL
);
GO

# **STATIC LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[RIVM_SUBVARIANT_MAPPING]'))
CREATE TABLE [VWSSTATIC].[RIVM_SUBVARIANT_MAPPING] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE(),
    [VARIANT_CODE] [VARCHAR] (100)  NULL,
    [VARIANT_NAME] [VARCHAR] (100)  NULL,
    [LABEL] [VARCHAR] (100)  NULL,
    [HISTORICALY_SIGNIFICANT] [BIT] DEFAULT(0) NULL
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_RIVM_SUBVARIANT_MAPPING]
	ON [VWSSTATIC].[RIVM_SUBVARIANT_MAPPING]
GO

CREATE INDEX [NCIX_DLI_RIVM_SUBVARIANT_MAPPING]
	ON [VWSSTATIC].[RIVM_SUBVARIANT_MAPPING] (
		[DATE_LAST_INSERTED]
	)
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_RIVM_SUBVARIANT_MAPPING]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[RIVM_SUBVARIANT_MAPPING] (
        [VARIANT_CODE],
        [VARIANT_NAME],
        [LABEL],
        [HISTORICALY_SIGNIFICANT]
    )
    SELECT
        [VARIANT_CODE],
        [VARIANT_NAME],
        [LABEL],
        [HISTORICALY_SIGNIFICANT]
    FROM
        [VWSSTAGE].[RIVM_SUBVARIANT_MAPPING]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_SUBVARIANT_MAPPING])
END;
GO